In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.mixture import GaussianMixture
import numpy as np

In [ ]:
import os
import torchvision.utils as vutils

# Create directories for saving images
real_image_dir = './data/real_images'
fake_image_dir = './data/fake_images'

os.makedirs(real_image_dir, exist_ok=True)
os.makedirs(fake_image_dir, exist_ok=True)

In [ ]:
transform = transforms.Compose([
    transforms.Resize(64),  # Resize to 64x64 to match generator output size
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torchvision.datasets import FashionMNIST
dataset  = FashionMNIST(root='./data', train=True, download=True, transform=transform)
subset_indices = torch.randperm(len(dataset))[:5000]
sub_dataset = Subset(dataset, subset_indices)

dataloader = DataLoader(sub_dataset, batch_size=128, shuffle=True)



In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 1, 4, 2, 1, bias=False),  # Output channel is 1 for grayscale
            nn.Tanh()  # Tanh activation to output values between -1 and 1
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),  # Input size: (64 x 64), Output size: (64 x 64)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 3, 2, 1),  # Input size: (64 x 64), Output size: (32 x 32)
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 3, 2, 1),  # Input size: (32 x 32), Output size: (16 x 16)
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 3, 2, 1),  # Input size: (16 x 16), Output size: (8 x 8)
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0),  # Input size: (8 x 8), Output size: (5 x 5)
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)




netG = Generator().to(device)
netD = Discriminator().to(device)



criterion = nn.BCELoss()
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [ ]:
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        # Train discriminator with real data
        netD.zero_grad()
        real_images, _ = data
        real_images = real_images.to(device)
        batch_size = real_images.size(0)
        label = torch.full((batch_size,), 1, device=device)
        output = netD(real_images).view(-1)
        errD_real = criterion(output, label.repeat(25).float())
        errD_real.backward()
        D_x = output.mean().item()

        # Train discriminator with fake data
        noise = torch.randn(batch_size, latent_size, 1, 1, device=device)
        fake_images = netG(noise)
        label.fill_(0)
        output = netD(fake_images.detach()).view(-1)
        errD_fake = criterion(output, label.repeat(25).float())
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        # Train generator
        netG.zero_grad()
        label.fill_(1)  # Fake labels are real for generator cost
        output = netD(fake_images).view(-1)
        errG = criterion(output, label.repeat(25).float())
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        # Print training stats
        if i % 100 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

    # Save generated images for every epoch
    if epoch == 0:
        torchvision.utils.save_image(real_images, './data/real_images.png', normalize=True)
    fake = netG(noise)
    torchvision.utils.save_image(fake.detach(), './data/fake_images_epoch_%03d.png' % epoch, normalize=True)

[0/30][0/40]	Loss_D: 1.3488	Loss_G: 1.2042	D(x): 0.5796	D(G(z)): 0.5357 / 0.3148


In [ ]:
for epoch in range(10):
    for i, data in enumerate(dataloader, 0):
        # Train discriminator with real data
        latent_size=100
        netD.zero_grad()
        real_images, _ = data
        real_images = real_images.to(device)
        batch_size = real_images.size(0)
        label = torch.full((batch_size,), 1, device=device)
        output = netD(real_images).view(-1)
        errD_real = criterion(output, label.repeat(25).float())
        errD_real.backward()
        D_x = output.mean().item()

        # Train discriminator with fake data
        noise = torch.randn(batch_size, latent_size, 1, 1, device=device)
        fake_images = netG(noise)
        label.fill_(0)
        output = netD(fake_images.detach()).view(-1)
        errD_fake = criterion(output, label.repeat(25).float())
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        # Train generator
        netG.zero_grad()
        label.fill_(1)  # Fake labels are real for generator cost
        output = netD(fake_images).view(-1)
        errG = criterion(output, label.repeat(25).float())
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        # Print training stats
        if i % 100 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, 10, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

    # Save generated images for every epoch
    if epoch == 0:
        torchvision.utils.save_image(real_images, './data/real_images.png', normalize=True)
    fake = netG(fixed_noise)
    torchvision.utils.save_image(fake.detach(), './data/fake_images_epoch_%03d.png' % epoch, normalize=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 4, 2, 1),
            nn.Tanh()
        )
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

In [ ]:
for epoch in range(10):
        for i, data in enumerate(dataloader, 0):
          # Update Discriminator
          netD.zero_grad()
          real_data = data[0].to(device)
          batch_size = real_data.size(0)
          label = torch.full((batch_size,), 1, device=device)
          output = netD(real_data)
          errD_real = criterionGAN(output, label)
          errD_real.backward()

          # Generate noise tensor
          noise = torch.randn(batch_size, 100, 1, 1, device=device)
          print("Noise shape:", noise.shape)

          fake_data = netG(noise)
          label.fill_(0)
          output = netD(fake_data.detach()).view(-1)
          errD_fake = criterionGAN(output, label)
          errD_fake.backward()
          errD = errD_real + errD_fake
          optimizerD.step()

          # Update Generator
          netG.zero_grad()
          label.fill_(1)
          output = netD(fake_data).view(-1)
          errG = criterionGAN(output, label)
          errG.backward()
          optimizerG.step()

          # Print statistics
          print(f"[{epoch}/{10}][{i}/{len(dataloader)}] "
                f"Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f}")